In [12]:
import numpy as np
import plotly.figure_factory as ff
import seaborn as sns

from PageRankSolver.pretty_print import matprint, plot_matrix
import PageRankSolver.dummy_solver


In [13]:
A = np.array([
              [0 , 0, 1, 0.5], 
              [1/3, 0, 0, 0,],
              [1/3, 1/2, 0, 1/2],
              [1/3, 1/2, 0, 0]
              ])
matprint(A, '.2f')

0.00  0.00  1.00  0.50  
0.33  0.00  0.00  0.00  
0.33  0.50  0.00  0.50  
0.33  0.50  0.00  0.00  
